In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("Sentiments.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'Sentiments.csv'

checking for Missing and Null Values

In [ ]:
label_counts=df["sentiment"].value_counts()
label_counts

In [ ]:
#create a bar plot
plt.figure(figsize=(5,5))
plt.bar(label_counts.index,label_counts.values,color=["red","blue","green"])
plt.xlabel("sentiment")
plt.ylabel("count")
plt.title("number of each label")
plt.show()

In [ ]:
#check for missing values
missing_values=df.isnull().sum()
missing_values

In [ ]:
blanks=[]
for i , email,label in df.itertuples():
    if email.isspace():
        blanks.append(i)
print(len(blanks),"|","blanks:",blanks)

In [ ]:
!pip install hazm

In [ ]:
import hazm

In [ ]:
stopwords = hazm.stopwords_list()  # Load Persian stopwords list
lemmatizer = hazm.Lemmatizer()  # Create a lemmatizer for reducing words to their root form
normalizer=hazm.Normalizer()  # Create a normalizer for standardizing Persian text

In [ ]:
print(len(stopwords))
stopwords[0:10]

In [ ]:
## Normalize Persian text to standardize spelling and spacing
test="سلام میکنم دوستان عزیز به خصوص آقای محمدی و محمدیان"
clean=normalizer.normalize(test)
clean

In [ ]:
def preprocess_text(messy_string):
  assert(type(messy_string)==str)

  #normlize,tokenize,remove stopwords lemmatize and rejoin
  cleaned=" ".join([
      lemmatizer.lemmatize(word)
      for word in normalizer.normalize(messy_string).split()
      if word not in stopwords
  ])
  return cleaned

In [ ]:
df["comment"]=df["comment"].apply(preprocess_text)
df.head()

preprocessing Data

In [ ]:
## Import tools for building a text processing pipeline and converting text to TF-IDF features
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer

In [ ]:
#Defind the pipeline with a more descriptive name
text_processing_pipeline=Pipeline([
    ("Vectorize",CountVectorizer()),## Change text into numbers by counting words
    ("TF-IDF",TfidfTransformer()) ## Change word counts to TF-IDF scores

])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df["comment"],df["sentiment"],test_size=0.2 , random_state=42)

In [ ]:
# Fit pipeline to training data and transform it; then transform test data
x_train = text_processing_pipeline.fit_transform(x_train)
x_test = text_processing_pipeline.transform(x_test)

training Models

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
#Define a dictionary of classifiers for essier refrencing and potential scalabing
classifiers={
    "SVC":SVC(kernel="linear"),
    "RandomForest":RandomForestClassifier(n_estimators=100 , random_state=42),
    "LogisticRegression":LogisticRegression(random_state=42),
    "MultinomialNB":MultinomialNB()

}
#iterate over each classifier and print report
for classifier_name , classifier_obj in classifiers.items():
  #fit the model
  classifier_obj.fit(x_train,y_train)

  #make predictions
  y_pred=classifier_obj.predict(x_test)

  #generate classifiction report
  report = classification_report(y_test,y_pred)

  #print the classifiercation report
  print(f"Classifiction report for {classifier_name}")
  print(report)
  print("----------------------------------------------")


In [ ]:
df.iloc[1600]

In [ ]:
#test the model with  custom email
#comment = preprocess_text ("من این فیلم رو دوست نداشتم و خیلی بدم اومد")

comment=df.iloc[1600]["comment"]
coustom_comment = text_processing_pipeline.transform([comment])
print(classifiers["SVC"].predict(coustom_comment))